In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

tf.executing_eagerly()

True

In [3]:
# thanks https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2021-08-30 14:10:16.853910: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-08-30 14:10:16.887982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 14:10:16.888276: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.095GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2021-08-30 14:10:16.888447: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-08-30 14:10:16.889607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-08-30 14:10:16.890729: I tensorflow/s

In [4]:
Pa = np.array([[0.71893144, 0.14144765, 0.10698649, 0.03263443],
       [0.06325843, 0.77423272, 0.13928361, 0.02322523],
       [0.02322523, 0.13928361, 0.77423272, 0.06325843],
       [0.03263443, 0.10698649, 0.14144765, 0.71893144]])

Pb = np.array([[0.95154866, 0.01674797, 0.0234783 , 0.00822507],
       [0.03863562, 0.84803839, 0.04732459, 0.0660014 ],
       [0.0660014 , 0.04732459, 0.84803839, 0.03863562],
       [0.00822507, 0.0234783 , 0.01674797, 0.95154866]])

Pc = np.array([[0.88137945, 0.04613594, 0.02686734, 0.04561727],
       [0.05009116, 0.9236776 , 0.0194023 , 0.00682894],
       [0.00682894, 0.0194023 , 0.9236776 , 0.05009116],
       [0.04561727, 0.02686734, 0.04613594, 0.88137945]])
Pm = np.array([[0.81203158, 0.04399917, 0.07848442, 0.06548484],
       [0.08284662, 0.87115553, 0.02024715, 0.0257507 ],
       [0.0257507 , 0.02024715, 0.87115553, 0.08284662],
       [0.06548484, 0.07848442, 0.04399917, 0.81203158]])

pi = np.array([0.14977394, 0.26317362, 0.39165429, 0.19539816])

pi_m = pi @ Pm

J = np.array([[[0.07508613297288518, 0.00466350788803318, 0.0033432901131033796, 0.004042219322757277], [0.002186937387234338, 0.01570096614179908, 0.0011254388766303509, 0.0005015574893929194], [0.002007283482847851, 0.0012535337368050612, 0.013395860029779497, 0.0010220788989917808], [0.0013011086711092986, 0.0016042909684947522, 0.0012404869820179004, 0.011323417297654589]], [[0.03129698648252512, 0.00815953485072002, 0.004394876926278819, 0.0020308813605868174], [0.008288184321098279, 0.14201837079233715, 0.005398759767993272, 0.0017880581744532536], [0.0015277493099248596, 0.00886482820182737, 0.04269081732021271, 0.0028169181161841477], [0.0021015052010260205, 0.011819352252411985, 0.0034103593745217937, 0.02379160092199082]], [[0.02174460265532043, 0.002941403304710777, 0.017006110729978843, 0.0023681733984058486], [0.0022577498981608663, 0.03237071808690972, 0.012426756488990255, 0.0019160961759307586], [0.0022144999652547693, 0.006239396030284971, 0.20953281339937846, 0.012130269098422072], [0.002543170970124578, 0.003973434481032667, 0.011775618913078878, 0.04210892174683528]], [[0.012093125924573445, 0.0012993605630256348, 0.0021464967072447926, 0.0015645368675293004], [0.0010702747292501203, 0.013479197587703494, 0.0016493822206027895, 0.0025896804160072057], [0.0005894417935184289, 0.00127918145334602, 0.022260289874459077, 0.0029430040436876307], [0.005229012301832341, 0.0040480932067737535, 0.006152859243062461, 0.09785943409093556]]])

In [5]:
J_mb = pi_m[:, np.newaxis] * Pb
J_bc = J_mb.T @ Pc
J_ab = Pa.T @ np.diag(pi) @ Pm @ Pb
J_ca = (Pa.T @ np.diag(pi) @ Pm @ Pc).T

assert np.allclose(J_bc, J.sum(axis=0))  # checking I have constructed it correctly
assert np.allclose(J_ab, J.sum(axis=2))
assert np.allclose(J_ca, J.sum(axis=1).T)

J_einsum = np.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
assert np.allclose(J_einsum, J)

In [9]:
# thanks https://github.com/mlgxmez/thelongrun_notebooks/blob/master/MLE_tutorial.ipynb

@tf.function()
def transform_P_matrix(params):
    params = tf.exp(params)
    Q0 = tf.concat([[-tf.reduce_sum(params[0])], params[0]],
                   axis=0)
    Q1 = tf.concat([[params[1,0]], [-tf.reduce_sum(params[1])], params[1,1:]],
                   axis=0)
    Q = tf.concat([[Q0], [Q1], [Q1[::-1]], [Q0[::-1]]], axis=0)
    return tf.linalg.expm(Q)

@tf.function()
def transform(params):
    pi = tfb.SoftmaxCentered()(params[0])
    Pa = transform_P_matrix(params[1:3])
    Pm = transform_P_matrix(params[3:5])
    Pb = transform_P_matrix(params[5:7])
    Pc = transform_P_matrix(params[7:9])
    return pi, Pa, Pm, Pb, Pc
    
@tf.function()
def loss(params, data):
    pi, Pa, Pm, Pb, Pc = transform(params)
    J = tf.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
    loss = tf.reduce_sum(tf.keras.losses.KLDivergence()(J, data))
    return loss

@tf.function()
def training_step(parameters, data, optimizer):
    with tf.GradientTape() as tape:
        loss_value = loss(parameters, data)
    gradients = tape.gradient(loss_value, parameters)
    optimizer.apply_gradients([(gradients, parameters)])
    return loss_value

def mle_run(data, loss, parameters, optimizer, tol=1e-9, steps=5000, verbose=False):
    update_list = []
    
    with tf.GradientTape() as tape:
        loss_value = loss(parameters, data)
    gradients = tape.gradient(loss_value, parameters)
    optimizer.apply_gradients([(gradients, parameters)])
    
    last_loss = None
    for i in range(steps):
        loss_value = training_step(parameters, data, optimizer)
    
        if i % 100 == 0:
            update_list.append((
                optimizer.iterations.numpy(),loss_value.numpy(), 
                parameters.numpy()))
            iter_info = f"Step: {optimizer.iterations.numpy()}, initial loss: {loss_value.numpy()}"
            if verbose:
                print(iter_info)
        
        if last_loss and np.abs(last_loss - loss_value) < tol:
            break
        last_loss = loss_value.numpy()
    
    return update_list

In [11]:
%%time
normal_initializer = tf.random_normal_initializer()
params = tf.Variable(normal_initializer(shape=[9, 3], dtype=tf.float32), name='params')

optimizer = tf.optimizers.Adam(0.01)
update_list = mle_run(J, loss, params, optimizer, verbose=True)

Step: 2, initial loss: 0.06802451610565186
Step: 102, initial loss: 0.054375916719436646
Step: 202, initial loss: 0.029549051076173782
Step: 302, initial loss: 0.024169042706489563
Step: 402, initial loss: 0.011589404195547104
Step: 502, initial loss: 0.004432496149092913
Step: 602, initial loss: 0.001679866574704647
Step: 702, initial loss: 0.0002588656498119235
Step: 802, initial loss: 9.796224185265601e-05
Step: 902, initial loss: 5.965631135040894e-05
Step: 1002, initial loss: 4.2164683691225946e-05
Step: 1102, initial loss: 3.200123319402337e-05
Step: 1202, initial loss: 2.5280700356233865e-05
Step: 1302, initial loss: 2.0458275685086846e-05
Step: 1402, initial loss: 1.684140443103388e-05
Step: 1502, initial loss: 1.399157554260455e-05
CPU times: user 11 s, sys: 1.68 s, total: 12.7 s
Wall time: 7.04 s


In [8]:
for p, phat in zip([pi, Pa, Pm, Pb, Pc], transform(params)):
    print(p)
    print(phat.numpy())
    print()

[0.14977394 0.26317362 0.39165429 0.19539816]
[0.15924823 0.25345227 0.37730977 0.20998974]

[[0.71893144 0.14144765 0.10698649 0.03263443]
 [0.06325843 0.77423272 0.13928361 0.02322523]
 [0.02322523 0.13928361 0.77423272 0.06325843]
 [0.03263443 0.10698649 0.14144765 0.71893144]]
[[0.66221833 0.19392754 0.117075   0.02677916]
 [0.06035676 0.7718665  0.13022181 0.03755496]
 [0.03755496 0.13022181 0.7718665  0.06035676]
 [0.02677916 0.11707501 0.19392754 0.66221833]]

[[0.81203158 0.04399917 0.07848442 0.06548484]
 [0.08284662 0.87115553 0.02024715 0.0257507 ]
 [0.0257507  0.02024715 0.87115553 0.08284662]
 [0.06548484 0.07848442 0.04399917 0.81203158]]
[[0.83121365 0.05142147 0.04501026 0.07235461]
 [0.04564698 0.904586   0.03204318 0.01772382]
 [0.01772382 0.03204317 0.904586   0.04564699]
 [0.07235461 0.04501025 0.05142147 0.8312136 ]]

[[0.95154866 0.01674797 0.0234783  0.00822507]
 [0.03863562 0.84803839 0.04732459 0.0660014 ]
 [0.0660014  0.04732459 0.84803839 0.03863562]
 [0.0082

In [17]:
params.initialized_value

<bound method Variable.initialized_value of <tf.Variable 'params:0' shape=(9, 3) dtype=float32, numpy=
array([[-0.27520323,  0.19507834,  0.5938708 ],
       [-1.4200332 , -1.9565469 , -3.630569  ],
       [-2.6245232 , -1.8489523 , -3.0636463 ],
       [-2.635978  , -3.087528  , -2.4449322 ],
       [-2.7804892 , -3.3084257 , -4.104154  ],
       [-3.9800875 , -3.7592027 , -4.8686886 ],
       [-3.1996157 , -2.9020796 , -2.628562  ],
       [-2.9895077 , -3.5608168 , -2.9907763 ],
       [-2.9099834 , -3.9275732 , -4.8003864 ]], dtype=float32)>>

In [18]:
params

<tf.Variable 'params:0' shape=(9, 3) dtype=float32, numpy=
array([[-0.27520323,  0.19507834,  0.5938708 ],
       [-1.4200332 , -1.9565469 , -3.630569  ],
       [-2.6245232 , -1.8489523 , -3.0636463 ],
       [-2.635978  , -3.087528  , -2.4449322 ],
       [-2.7804892 , -3.3084257 , -4.104154  ],
       [-3.9800875 , -3.7592027 , -4.8686886 ],
       [-3.1996157 , -2.9020796 , -2.628562  ],
       [-2.9895077 , -3.5608168 , -2.9907763 ],
       [-2.9099834 , -3.9275732 , -4.8003864 ]], dtype=float32)>